In [ ]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            driver.quit() # Close the driver

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('telangana_bus_details.csv', index=False)


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/hyderabad-to-warangal: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF766316CB5+28821]
	(No symbol) [0x00007FF766283840]
	(No symbol) [0x00007FF76612578A]
	(No symbol) [0x00007FF7661791BE]
	(No symbol) [0x00007FF7661794AC]
	(No symbol) [0x00007FF7661C2647]
	(No symbol) [0x00007FF76619F33F]
	(No symbol) [0x00007FF7661BF412]
	(No symbol) [0x00007FF76619F0A3]
	(No symbol) [0x00007FF76616A778]
	(No symbol) [0x00007FF76616B8E1]
	GetHandleVerifier [0x00007FF76664FCAD+3408013]
	GetHandleVerifier [0x00007FF76666741F+3504127]
	GetHandleVerifier [0x00007FF76665B5FD+3455453]
	GetHandleVerifier [0x00007FF7663DBDBB+835995]
	(No symbol) [0x00007FF76628EB5F]
	(No symbol) [0x00007FF76628A814]
	(No symbol) [0x00007FF76628A9AD]
	(No symbol) [0x00007FF76627A199]
	BaseThreadInitThunk [0x00007FFF516B259D+29]
	RtlUserThreadStart [0x00007FFF5224AF38+40]

Scraping completed. Data saved to telangana_bus_details.csv


In [7]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8935,Super Luxury (Non AC Seater 2+2 Push Back),00:20,03h 50m,04:10,3.7,INR 320,16 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8987,Super Luxury (Non AC Seater 2+2 Push Back),00:25,03h 55m,04:20,3.6,INR 320,18 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 34333,LAHARI A/C SLEEPER CUM SEATER,00:50,04h 10m,05:00,3.1,INR 389,30 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8949,RAJDHANI (A.C. Semi Sleeper),01:15,03h 55m,05:10,2.2,INR 389,27 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8885,Lahari Non A/C Sleeper Cum Seater,01:15,04h 00m,05:15,4.4,INR 320,5 Seats available
...,...,...,...,...,...,...,...,...,...,...
1507,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Go Tour Travels and Holidays,A/C Seater / Sleeper (2+1),23:10,03h 20m,02:30,4.3,INR 690,21 Seats available
1508,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:30,05h 15m,04:45,3.2,332,37 Seats available
1509,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:50,05h 25m,05:15,3.6,379,34 Seats available
1510,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),06:10,04h 20m,10:30,2.7,INR 500,25 Seats available


In [7]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8935,Super Luxury (Non AC Seater 2+2 Push Back),00:20,03h 50m,04:10,3.7,INR 320,16 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8987,Super Luxury (Non AC Seater 2+2 Push Back),00:25,03h 55m,04:20,3.6,INR 320,18 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 34333,LAHARI A/C SLEEPER CUM SEATER,00:50,04h 10m,05:00,3.1,INR 389,30 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8949,RAJDHANI (A.C. Semi Sleeper),01:15,03h 55m,05:10,2.2,INR 389,27 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TGSRTC - 8885,Lahari Non A/C Sleeper Cum Seater,01:15,04h 00m,05:15,4.4,INR 320,5 Seats available
...,...,...,...,...,...,...,...,...,...,...
1507,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Go Tour Travels and Holidays,A/C Seater / Sleeper (2+1),23:10,03h 20m,02:30,4.3,INR 690,21 Seats available
1508,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:30,05h 15m,04:45,3.2,332,37 Seats available
1509,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,NueGo,Electric A/C Seater (2+2),23:50,05h 25m,05:15,3.6,379,34 Seats available
1510,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...,Morning Star Travels,A/C Sleeper (2+1),06:10,04h 20m,10:30,2.7,INR 500,25 Seats available
